# F1 2025 - Prédiction des classements finaux

Ce notebook a pour objectif de prédire les grilles de départ et les positions finales des pilotes pour la saison 2025 de Formule 1.

Pour cela, nous utilisons des données historiques des saisons 1950 à 2024, enrichies de variables spécifiques à la saison 2025, telles que le type de circuit, les conditions météorologiques prévues, ou encore le statut de course à domicile pour chaque pilote.

Deux modèles de régression distincts sont construits :
- Le premier vise à prédire la **position sur la grille de départ** ;
- Le second anticipe la **position finale à l’arrivée**.

Ces modèles sont ensuite appliqués à l’ensemble du calendrier 2025, permettant d’obtenir des prédictions détaillées pour chaque course.

## Table des matières

1. [Chargement des bibliothèques](#chargement-des-bibliothèques)  
2. [Chargement et filtrage des données historiques](#chargement-et-filtrage-des-données-historiques)  
3. [Modélisation de la grille de départ](#modélisation-de-la-grille-de-départ)  
4. [Modélisation de la position finale](#modélisation-de-la-position-finale)  
5. [Préparation des données de la saison 2025](#préparation-des-données-de-la-saison-2025)  
6. [Prédictions pour la saison 2025](#prédictions-pour-la-saison-2025)  
7. [Affichage des résultats par course](#affichage-des-résultats-par-course)  
8. [Sauvegarde des prédictions](#sauvegarde-des-prédictions)  
9. [Classement moyen pilotes / écuries](#classement-moyen-pilotes--écuries)  
10. [Conclusion](#conclusion)


## Chargement des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

## Chargement des données historiques récentes (≥ 2020)

Afin d’assurer une meilleure cohérence avec le contexte actuel de la Formule 1, nous filtrons les données historiques à partir de la saison 2020. Cette période récente reflète mieux les conditions modernes du championnat, avec des changements significatifs dans les règlements, l’arrivée de nouveaux circuits et l’évolution des performances des écuries.
Cela permet d’entraîner des modèles plus adaptés aux caractéristiques de la saison 2025, en réduisant l’influence de saisons trop anciennes et moins représentatives.

In [2]:
df_hist = pd.read_csv("../data/data_filter.csv")
df_hist = df_hist[df_hist["year"] >= 2020]  # ou >= 2018 ajuster selon les besoins

# Supprimer les colonnes manquantes attendues si elles n'existent pas
for col in ["circuit_type", "laps_mean_time", "weather_profile", "team_points_last_y"]:
    if col not in df_hist.columns:
        df_hist[col] = np.nan  # Valeur par défaut temporaire ou 0.0 si pertinent
        if col == "team_points_last_y":
            df_hist[col] = 0.0

C:\Users\rapha\AppData\Local\Temp\ipykernel_11368\3112964031.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hist = pd.read_csv("../data/data_filter.csv")


## Prédiction de la position sur la grille de départ

In [3]:
features_grid = [
    "driver_age", "constructor_name", "races_name", "driver_nationality_1",
    "circuit_type", "laps_mean_time", "weather_profile", "team_points_last_y",
    "is_home_race"
]
target_grid = "grid"

Xg = df_hist[features_grid]
yg = df_hist[target_grid]

categorical_cols_grid = ["constructor_name", "races_name", "driver_nationality_1", "circuit_type", "weather_profile"]
numeric_cols_grid = ["driver_age", "laps_mean_time", "team_points_last_y","is_home_race"]

preprocessor_grid = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols_grid),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols_grid),
])

pipeline_grid = Pipeline([
    ("preprocessor", preprocessor_grid),
    ("regressor", HistGradientBoostingRegressor())
])

Xg_train, Xg_val, yg_train, yg_val = train_test_split(Xg, yg, test_size=0.3, random_state=42)
pipeline_grid.fit(Xg_train, yg_train)

yg_pred = pipeline_grid.predict(Xg_val)
print("MAE grille de départ :", mean_absolute_error(yg_val, yg_pred))

c:\Users\rapha\anaconda3\envs\f1-ml\lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\rapha\anaconda3\envs\f1-ml\lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\rapha\anaconda3\envs\f1-ml\lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


MAE grille de départ : 3.3760708124475705


Ce bloc permet de prédire la position de départ (grid) d’un pilote avant la course, à partir de plusieurs informations connues en amont : âge, écurie, nom du circuit, météo, etc.

Un pipeline est utilisé pour :

- Standardiser les données numériques,

- Encoder les données catégorielles,

- Entraîner un modèle de type HistGradientBoostingRegressor.

Le modèle est évalué avec une division entraînement/validation et l’erreur moyenne absolue (MAE) est calculée.

## Prédiction de la position finale à l’arrivée

In [4]:
features_final = features_grid.copy()
features_final.insert(0, "grid")  # On ajoute la grille comme premier facteur

Xf = df_hist[features_final]
yf = df_hist["positionOrder"]

categorical_cols_final = categorical_cols_grid.copy()
numeric_cols_final = numeric_cols_grid.copy()
numeric_cols_final.insert(0, "grid")

preprocessor_final = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols_final),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols_final),
])

pipeline_final = Pipeline([
    ("preprocessor", preprocessor_final),
    ("regressor", HistGradientBoostingRegressor())
])

Xf_train, Xf_val, yf_train, yf_val = train_test_split(Xf, yf, test_size=0.3, random_state=42)
pipeline_final.fit(Xf_train, yf_train)

yf_pred = pipeline_final.predict(Xf_val)
print("MAE position finale :", mean_absolute_error(yf_val, yf_pred))

c:\Users\rapha\anaconda3\envs\f1-ml\lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\rapha\anaconda3\envs\f1-ml\lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\rapha\anaconda3\envs\f1-ml\lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


MAE position finale : 3.3898347541657317


Ce bloc permet de prédire la position finale (positionOrder) d’un pilote à la fin de la course, en ajoutant la position sur la grille de départ comme nouvelle variable explicative.

Nous utilisons :

- Les mêmes variables que pour la prédiction de la grille (features_grid),

- En y ajoutant la grille de départ (grid) comme facteur supplémentaire.

Le pipeline inclut :

- Un prétraitement des variables numériques et catégorielles,

- Un modèle de régression (HistGradientBoostingRegressor),

- Une évaluation sur un jeu de validation à l’aide de la MAE.

Cette étape vise à mesurer la capacité du modèle à anticiper le classement final à partir de données de départ.

## Prédiction des résultats pour la saison 2025

In [5]:
df_2025 = pd.read_csv("../data/season2025/data_2025.csv")
df_2025 = df_2025.drop_duplicates(subset=["circuit_name", "driver_name"], keep="first")
df_2025.rename(columns={"circuit_name": "races_name"}, inplace=True)

# Prédire la grille de départ
X_2025_grid = df_2025[features_grid]
df_2025["grid"] = pipeline_grid.predict(X_2025_grid)

# Prédire la position finale
X_2025_final = df_2025[features_final]
df_2025["predicted_positionOrder"] = pipeline_final.predict(X_2025_final)


Dans ce bloc, on applique les modèles entraînés pour prédire les performances des pilotes en 2025 :

- Chargement des données 2025, avec suppression des doublons pilotes/circuits.

- Prévision de la grille de départ avec le modèle pipeline_grid.

- La grille prédite est ensuite utilisée comme variable d'entrée pour prédire la position finale avec le modèle pipeline_final.

## Affichage des résultats par course : grille vs position finale prédite

In [6]:
def afficher_resultats_course(df, course_name):
    df_course = df[df["races_name"] == course_name].copy()

    # Grille de départ
    df_grid = df_course.sort_values(by="grid").reset_index(drop=True)
    df_grid["grid_rank"] = df_grid.index + 1
    print(f"\n=== 🇫🇷 Grille de départ - {course_name} ===")
    for _, row in df_grid.iterrows():
        print(f"P{int(row['grid_rank']):>2} - {row['driver_name']} ({row['constructor_name']})")

    # Position finale
    df_sorted = df_course.sort_values(by="predicted_positionOrder").reset_index(drop=True)
    df_sorted["rang_final"] = df_sorted.index + 1
    print(f"\n=== Position finale prédite - {course_name} ===")
    for _, row in df_sorted.iterrows():
        print(f"P{int(row['rang_final']):>2} - {row['driver_name']} ({row['constructor_name']})")

## Choix manuel d'une course à afficher

Ce bloc permet de sélectionner manuellement une course (par son nom) afin d’afficher les résultats prédits de la grille de départ et du classement final, calculer au dessus.

In [7]:
afficher_resultats_course(df_2025, "Monaco (Monte-Carlo)")


=== 🇫🇷 Grille de départ - Monaco (Monte-Carlo) ===
P 1 - George Russell (Mercedes)
P 2 - Charles Leclerc (Ferrari)
P 3 - Max Verstappen (Red Bull)
P 4 - Yuki Tsunoda (Red Bull)
P 5 - Andrea Kimi Antonelli (Mercedes)
P 6 - Lando Norris (McLaren)
P 7 - Lewis Hamilton (Ferrari)
P 8 - Oscar Piastri (McLaren)
P 9 - Pierre Gasly (Alpine)
P10 - Carlos Sainz (Williams)
P11 - Alexander Albon (Williams)
P12 - Liam Lawson (Racing Bulls)
P13 - Esteban Ocon (Haas F1 Team)
P14 - Lance Stroll (Aston Martin)
P15 - Fernando Alonso (Aston Martin)
P16 - Nico Hulkenberg (Kick Sauber)
P17 - Gabriel Bortoleto (Kick Sauber)
P18 - Isack Hadjar (Racing Bulls)
P19 - Franco Colapinto (Alpine)
P20 - Oliver Bearman (Haas F1 Team)

=== Position finale prédite - Monaco (Monte-Carlo) ===
P 1 - George Russell (Mercedes)
P 2 - Charles Leclerc (Ferrari)
P 3 - Max Verstappen (Red Bull)
P 4 - Yuki Tsunoda (Red Bull)
P 5 - Andrea Kimi Antonelli (Mercedes)
P 6 - Oscar Piastri (McLaren)
P 7 - Lewis Hamilton (Ferrari)
P 8 - 

On observe plusieurs changements notables entre la grille de départ et la position finale prédite :

- Yuki Tsunoda passe de la 3ᵉ à la 1ʳᵉ place, suggérant une excellente performance en course.

- George Russell, parti en pole position, termine 4ᵉ.

- Charles Leclerc, bien placé dès le départ (4ᵉ), parvient à monter sur le podium (2ᵉ).

- Certains pilotes comme Nico Hülkenberg ou Gabriel Bortoleto réalisent de belles remontées depuis le fond de grille.

- À l’inverse, Alexander Albon (10ᵉ sur la grille) et Esteban Ocon (11ᵉ) chutent significativement au classement final.

Ces résultats traduisent des performances différenciées selon les pilotes et les écuries, et illustrent l’intérêt d’utiliser la grille de départ comme variable prédictive dans le modèle. Bien sur, les résultats sont pas encore parfait mais c'est déja un bon début.

## Afficher toutes les grilles et positions finales prédictives de 2025

In [8]:
for gp in sorted(df_2025["races_name"].unique()):
    afficher_resultats_course(df_2025, gp)


=== 🇫🇷 Grille de départ - Abou Dhabi (Yas Marina) ===
P 1 - George Russell (Mercedes)
P 2 - Max Verstappen (Red Bull)
P 3 - Yuki Tsunoda (Red Bull)
P 4 - Charles Leclerc (Ferrari)
P 5 - Andrea Kimi Antonelli (Mercedes)
P 6 - Lando Norris (McLaren)
P 7 - Lewis Hamilton (Ferrari)
P 8 - Oscar Piastri (McLaren)
P 9 - Pierre Gasly (Alpine)
P10 - Carlos Sainz (Williams)
P11 - Liam Lawson (Racing Bulls)
P12 - Alexander Albon (Williams)
P13 - Lance Stroll (Aston Martin)
P14 - Esteban Ocon (Haas F1 Team)
P15 - Fernando Alonso (Aston Martin)
P16 - Nico Hulkenberg (Kick Sauber)
P17 - Isack Hadjar (Racing Bulls)
P18 - Gabriel Bortoleto (Kick Sauber)
P19 - Franco Colapinto (Alpine)
P20 - Oliver Bearman (Haas F1 Team)

=== Position finale prédite - Abou Dhabi (Yas Marina) ===
P 1 - George Russell (Mercedes)
P 2 - Max Verstappen (Red Bull)
P 3 - Charles Leclerc (Ferrari)
P 4 - Oscar Piastri (McLaren)
P 5 - Yuki Tsunoda (Red Bull)
P 6 - Lewis Hamilton (Ferrari)
P 7 - Lando Norris (McLaren)
P 8 - Andr

## Sauvegarde des prédictions

Les résultats finaux, incluant la grille de départ et la position finale prédite pour chaque pilote, sont enregistrés dans un fichier CSV pour une consultation ou une analyse ultérieure.

In [9]:
df_2025.to_csv("../data/season2025/predictions_2025.csv", index=False)

## Classement moyen des pilotes et des écuries

In [10]:
print("\n Moyenne des positions finales prédites par pilote :")
print(df_2025.groupby("driver_name")["predicted_positionOrder"].mean().sort_values())

print("\n Moyenne des positions finales prédites par écurie :")
print(df_2025.groupby("constructor_name")["predicted_positionOrder"].mean().sort_values())


 Moyenne des positions finales prédites par pilote :
driver_name
George Russell            5.728057
Max Verstappen            6.943902
Charles Leclerc           7.331004
Yuki Tsunoda              7.779820
Oscar Piastri             7.859842
Lewis Hamilton            9.112231
Andrea Kimi Antonelli     9.215913
Lando Norris              9.370952
Pierre Gasly             10.535948
Nico Hulkenberg          11.693808
Liam Lawson              12.134502
Franco Colapinto         12.337693
Lance Stroll             12.458570
Gabriel Bortoleto        12.821764
Isack Hadjar             12.842612
Fernando Alonso          13.522693
Jack Doohan              14.486468
Carlos Sainz             15.212750
Alexander Albon          15.930598
Oliver Bearman           16.880674
Esteban Ocon             17.176581
Name: predicted_positionOrder, dtype: float64

 Moyenne des positions finales prédites par écurie :
constructor_name
Red Bull         7.210097
Mercedes         7.471985
Ferrari          8.221618
McLa

Ce bloc calcule la moyenne des positions finales prédites sur l’ensemble des courses :

- Par pilote, pour estimer la régularité individuelle.

- Par écurie, en prenant la moyenne des pilotes de chaque team.

Les valeurs les plus basses correspondent aux meilleures performances moyennes.

## Conclusion

Ce notebook présente un pipeline complet de régression visant à prédire les résultats de la saison 2025 de Formule 1.

Les principales étapes réalisées sont :

- L'entraînement de deux modèles distincts : l’un pour estimer la position sur la grille de départ, l’autre pour prédire le classement final.

- L’exploitation de données historiques enrichies pour simuler l’ensemble de la saison à venir.

- L’analyse comparative des performances moyennes des pilotes et des écuries.

- L’observation des écarts entre la grille et les résultats finaux, course par course.

Ce modèle constitue une première base de prévision des performances en F1, permettant d’identifier les pilotes et écuries les plus performants selon les simulations.
Des pistes d'amélioration incluent l'intégration d’éléments dynamiques comme les abandons, les stratégies de course ou la météo.
Enfin, l’ajout progressif des résultats réels de la saison 2025 permettra d’affiner les prédictions tout au long de l’année.
